# Gradient boosting

## Boosting

Michael Kearns (1988), [Thoughts on Hypothesis Boosting](https://www.cis.upenn.edu/~mkearns/papers/boostnote.pdf)

> In this paper we present initial and modest progress on the Hypothesis Boosting Problem. Informally, this problem asks whether an efficient learning algorithm that outputs an hypothesis whose performance is **only slightly better than random guessing** implies the existence of an efficient algorithm that outputs an hypothesis of arbitrary accuracy.

**“약한 학습기(weak learner)를 모아서 강한 학습기(strong learner)를 만들 수 있을까?”**

## Gradient boosting

**Q**. 현재 고정된 모델 $F_m$이 있다고 하고, 이를 이용해서 더 좋은 모델 $F_{m + 1}$을 만드는 방법엔 뭐가 있을까?

**A**. $F_m(x_i) + h_m(x_i) = y_i$가 되도록, 다시 말해 **잔차 $y_i - F_m(x_i)$를 예측하는 $h_m$을 학습해서 기존 모델에 더해** 새로운 모델을 만든다.

### Gradient descent와의 관계?

<div class="alert alert-info">
    <strong>참고.</strong>
    Gradient descent: 파라미터 $\theta_i$들과, 비용 함수 $L$이 있을 때, 파라미터를 기울기의 역방향으로 이동시킴
    $$
        \theta_i \leftarrow \theta_i - \rho \frac{\partial L}{\partial \theta_i}
    $$
</div>

비용 함수가 $L = \sum_{i} \frac12 (y_i - F(x_i))^2$일 때(MSE), $F(x_i)$를 파라미터라고 보고(?) 기울기를 취하면
$$
    \frac{\partial L}{\partial F(x_i)} = \frac{\partial \sum_{j} \frac12 (y_j - F(x_j))^2}{\partial F(x_i)} = \frac{\partial \left( \frac12 (y_i - F(x_i))^2 \right)}{\partial F(x_i)} = -(F(x_i) - y_i)
$$
가 되고, **잔차가 비용 함수의 기울기의 역방향**이 됩니다.

# XGBoost

- <https://xgboost.ai/>
- [arXiv:1603.02754 \[cs.LG\]](https://arxiv.org/abs/1603.02754)

다음과 같이 $K$개의 함수(트리)를 **더해서** 앙상블 모델을 만듭니다.

$$
    \hat{y}_i = \phi(\mathbf{x}_i) = \sum_{k = 1}^K f_k(\mathbf{x}_i) \tag{1}
$$

- $\mathcal{D} = \{ (\mathbf{x}_i, y_i) \}$ 데이터셋 ($\mathbf{x}_i \in \mathbb{R}^m$, $y_i \in \mathbb{R}$)
    - $n = \lvert \mathcal{D} \rvert$: 데이터 수
    - $m$: 피처 수
- $K$: 총 트리 개수

![Figure 1](figures/xgboost-figure-01.png)

비용(목표) 함수는 다음과 같습니다.
$$
    \mathcal{L}(\phi) = \sum_i l(y_i, \hat{y}_i) + \sum_k \Omega(f_k) \tag{2}
$$

- $l$: 미분 가능한 볼록 비용 함수
- $\Omega(f) = \gamma T + \frac12 \lambda \lVert w \rVert^2$: 모델의 복잡도에 따른 규제 함수. 잎 노드의 수가 많지 않게, 무게의 절댓값이 크지 않게 제약

목표 함수 (2)는 트리 자체가 식 안에 들어가 있기 때문에 기존 방식으로는 최적화하는 것이 불가능합니다.
그 대신, 트리를 계속 더해가는 방식(additive manner)으로 모델을 학습합니다.
변수 $\hat{y}_i^{(t)}$를 $t$번째 트리까지 이용한 $i$번째 데이터의 예측값이라 하면, 다음 목표 함수를 최소화하는 $f_t$를 찾아야 합니다.
$$
    \mathcal{L}^{(t)} = \sum_{i = 1}^n l(y_i, \hat{y}_i^{(t - 1)} + f_t(\mathbf{x}_i)) + \Omega(f_t)
$$

가능한 모든 트리 구조 $q$를 모두 탐색하는 것은 불가능하기 때문에, 하나의 잎 노드만 있는 트리에서 시작해서 탐욕 알고리즘으로 가지치기해 나가는 방법을 사용합니다.